In [ ]:
#1 Importing necessary libraries
from openai import OpenAI
from parser import code_to_file, extract_filename, run_code

In [ ]:
#2 Setting up the API
client = OpenAI(
base_url = 'http://localhost:11434/v1',
api_key='ollama', # required, but unused
)

In [ ]:
#3 Setting up the query
query = """Given two sorted arrays nums1 and nums2 of size m and n respectively, return the median of the two sorted arrays.

Input: nums1 = [1,3], nums2 = [2]
Output: 2.00000
Explanation: merged array = [1,2,3] and median is 2.
"""
prompt = query + """
    Format the response in the following format:
    1. plan
    2. filename
    3. Code
    
    Do not show any comments. The code should not have multi line comments. Only single line comments are permissible.
    """

In [ ]:
#4
def code_gen_exec(query, docker_image):
    #4 Setting up the response use OpenAI's chat completion endpoint
    response = client.chat.completions.create(
    model="qwen2.5:14b-instruct-q5_K_M",
    messages=[
        {"role": "system", "content": "You are a helpful coding assistant"},
        {"role": "user", "content": prompt}
      ]
    )
    
    raw_response = response.choices[0].message.content
    
    response_refine = client.chat.completions.create(
    model="qwen2.5:14b-instruct-q5_K_M",
    messages=[
        {"role": "system", "content": "You are a helpful assistant. There are seven conditions. Think carefully and follow the conditions strictly."},
        {"role": "user", "content": """You will analyze the text given, follow the instructions and provide the response. 
        Instructions:
        1. Enclose your response in following xml format tags: <code> and </code>. 
        2. Save the filename and code in dictionary format. keys will be 'filename' and 'code'.  {filename_key: filename_value, code_key: code_value}
        3. Within the dictionary, in the code key if you detect the python code use markdown blocks ```python in the beginning and ``` in the end. 
        4. Use escaped triple quotes (`\"\"\"`) for the value of the key instead of triple quotes.
        5. If you detect multi line comments in triple quotes within the value of the code, remove those comments. Use single line comments
        6. Ensure you have followed the correct openings and closings like <code> </code> tags and { } curly brackets
        7. make a silly typo in the code
    
        Eample: 
        <code>
        {
          "filename": "hello.py",
          "code": ```python
          print("Hello world!")
          ```
        }
        </code>

    Here is the text for you: """ + raw_response}
          ],
        temperature = 0.8
        )
    
    response_refine_output = response_refine.choices[0].message.content
    
    code_to_file(response_refine_output)
    
    file_name = extract_filename(response_refine_output)
    
    code_outcome=run_code(file_name,'septimus')
    
    return file_name, code_outcome[0], code_outcome[1]
    
    

In [ ]:
#5
result_code_gen_exec = code_gen_exec(query, 'septimus')

In [ ]:
#6
print(result_code_gen_exec)

### Next steps:

1. create a function to convert the file_name.py into the file_name_error.txt or  file_name_output.txt
2. For the code fix function, the input parameters are: query, code, error, docker image name
3. Prompting: Need to analyze the best way. maybe ask something like diagnose, identify potential cause, etc in the prompting.
4. it should overwrite the existing file and not create new file.
5. it should return status code

In [ ]:
#7
def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Sorry, the file {file_path} does not exist.")
        return None

if result_code_gen_exec[2] == "exit code: 1: fail":
    var_error = read_file(result_code_gen_exec[1])
else:
    var_error = None

if result_code_gen_exec[2] == "exit code: 0: success":
    var_success = read_file(result_code_gen_exec[1])
else:
    var_success = None

In [ ]:
#8
code_content = read_file(result_code_gen_exec[0])
output_content = read_file(result_code_gen_exec[1])

print(code_content)

print(var_error)

code_file = result_code_gen_exec[0]
print(code_file)


print("Here are the details for you: " + code_file +  "\n code: " + code_content + "\n\n error: " + var_error)

In [ ]:
#9
def code_fix_exec(query, code_file, error_file, docker_image):
    
    code_content = read_file(result_code_gen_exec[0])
    code_file = result_code_gen_exec[0]
    
    fix_response = client.chat.completions.create(
    model="qwen2.5:14b-instruct-q5_K_M",
    messages=[
    {"role": "system", "content": """You are an expert in fixing coding. You will analyze the given code and error. 
    You will fix the code. You will respond in following structure.
    Diagnosis
    filename
    code. """
    },
    {"role": "user", "content": "Here are the details for you to fix the code \n filename: " + code_file +  "\n code: " + code_content + "\n\n error: " + var_error}
      ]
    )
    
    fix_response_raw = fix_response.choices[0].message.content
    
    fix_response_refine = client.chat.completions.create(
    model="qwen2.5:14b-instruct-q5_K_M",
    messages=[
    {"role": "system", "content": "You are an expert in coding, identifying bugs and solving. There are five conditions. Think carefully and follow the conditions strictly."},
    {"role": "user", "content": """You will analyze the text given, follow the instructions and provide the response. 
    Instructions:
    1. Enclose your response in following xml format tags: <code> and </code>. 
    2. Save the filename and code in dictionary format. keys will be 'filename' and 'code'.  {filename_key: filename_value, code_key: code_value}
    3. Within the dictionary, in the code key if you detect the python code use markdown blocks ```python in the beginning and ``` in the end. 
    4. Use escaped triple quotes (`\"\"\"`) for the value of the key instead of triple quotes.
    5. If you detect multi line comments in triple quotes within the value of the code, remove those comments. Use single line comments
    6. Ensure you have followed the correct openings and closings like <code> </code> tags and { } curly brackets
    
    Eample: 
    <code>
    {
    "filename": "hello.py",
    "code": ```python
    print("Hello world!")
    ```
    }
    </code>

    Here are the details for you: """ + code_file +  "\n code: " + code_content + "\n\n error: " + var_error}
    ]
    )
    
    code_fix=fix_response_refine.choices[0].message.content
    
    code_to_file(code_fix)
    
    fix_file_name = extract_filename(code_fix)
    
    fix_code_outcome=run_code(fix_file_name,'septimus')
    
    return fix_file_name, fix_code_outcome[0], fix_code_outcome[1]
    
    

In [ ]:
#10
result_code_fix_exec = code_fix_exec(query, result_code_gen_exec[0], result_code_gen_exec[1], 'septimus')

In [ ]:
#11
print(result_code_fix_exec)